In [ ]:
!pip install gradio librosa matplotlib ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import librosa
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from IPython.display import Audio, display

def audio_equalizer(audio_file):
    """
    Takes an audio file, visualizes it as an equalizer, and provides a play option.

    Args:
        audio_file: Gradio Audio component's output (tuple of (sample_rate, audio_data)).

    Returns:
        HTML string containing the equalizer visualization and play button.
    """
    if audio_file is None:
        return "Please upload an audio file."

    sample_rate, audio_data = audio_file

    try:
        # Calculate Short-Time Fourier Transform (STFT)
        stft = librosa.stft(audio_data)
        magnitude = np.abs(stft)

        # Calculate average magnitude in frequency bands (adjust bands as needed)
        bands = [0, 100, 200, 400, 800, 1600, 3200, 6400, sample_rate // 2]  # Example bands
        band_magnitudes = []
        for i in range(len(bands) - 1):
            lower_bin = int(bands[i] * (stft.shape[0] - 1) / (sample_rate / 2))
            upper_bin = int(bands[i + 1] * (stft.shape[0] - 1) / (sample_rate / 2))
            band_magnitudes.append(np.mean(magnitude[lower_bin:upper_bin], axis=1))

        # Create equalizer visualization
        plt.figure(figsize=(10, 4))
        bar_width = 0.8
        x_positions = np.arange(len(bands) - 1)
        for i, band_mag in enumerate(band_magnitudes):
            plt.bar(x_positions[i], np.mean(band_mag), width=bar_width)

        plt.xticks(x_positions, [f"{bands[i]}-{bands[i+1]} Hz" for i in range(len(bands) - 1)])
        plt.xlabel("Frequency Bands")
        plt.ylabel("Average Magnitude")
        plt.title("Audio Equalizer")
        plt.tight_layout()

        # Save the plot to a BytesIO object
        img_buf = BytesIO()
        plt.savefig(img_buf, format='png')
        img_buf.seek(0)
        img_base64 = base64.b64encode(img_buf.getvalue()).decode('utf-8')
        plt.close()

        # Create HTML for the play button
        audio_html = Audio(audio_data, rate=sample_rate)._repr_html_()

        # Combine image and audio HTML
        html_output = f'<img src="data:image/png;base64,{img_base64}" alt="Equalizer"><br>{audio_html}'

        return html_output

    except Exception as e:
        return f"An error occurred: {e}"

iface = gr.Interface(
    fn=audio_equalizer,
    inputs=gr.Audio(type="numpy"),
    outputs=gr.HTML(),
    title="Audio Equalizer Visualizer",
    description="Upload an audio file to visualize it as an equalizer and play it."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84d0923c66292ce647.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
